<a href="https://colab.research.google.com/github/29aathira/MSc-Dissertation/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df.values

data = load_data('/content/sales_train_evaluation.csv')

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse, binary_crossentropy

# Define input dimension
input_dim = 1947

# Define latent space dimension
latent_dim = 10  # You can adjust this based on your requirements

# Encoder network
inputs = Input(shape=(input_dim,))
enc = Dense(512, activation='relu')(inputs)
enc_mean = Dense(latent_dim)(enc)
enc_log_var = Dense(latent_dim)(enc)

# Reparameterization trick
def sampling(args):
    enc_mean, enc_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(enc_mean)[0], latent_dim))
    return enc_mean + tf.exp(0.5 * enc_log_var) * epsilon

latent_vector = Lambda(sampling)([enc_mean, enc_log_var])

# Decoder network
dec = Dense(512, activation='relu')(latent_vector)
outputs = Dense(input_dim)(dec)

# VAE model
vae = Model(inputs, outputs)

# Define VAE loss function
def vae_loss(inputs, outputs):
    reconstruction_loss = mse(inputs, outputs)
    kl_loss = -0.5 * tf.reduce_mean(1 + enc_log_var - tf.square(enc_mean) - tf.exp(enc_log_var))
    return reconstruction_loss + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)

# Train VAE
vae.fit(train_data, train_data, epochs=50, batch_size=32, validation_data=(val_data, val_data))


ValueError: ignored

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

# Load data
def load_data():
    df = pd.read_csv('/content/sales_train_evaluation.csv')
    return df.values

data = load_data()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse, KLD
from tensorflow.keras import backend as K

# Load data
def load_data():
    df = pd.read_csv('/content/sales_train_evaluation.csv')
    return df.values

data = load_data()

# Convert string values to integers
def preprocess_data(data):
    label_encoders = []
    for col_idx in range(data.shape[1]):
        if isinstance(data[0, col_idx], str):
            le = LabelEncoder()
            data[:, col_idx] = le.fit_transform(data[:, col_idx])
            label_encoders.append(le)
    return data, label_encoders

data, label_encoders = preprocess_data(data)

# Split data into train and validation sets
def split_data(data):
    train_size = int(0.8 * len(data))
    train_data = data[:train_size]
    val_data = data[train_size:]
    return train_data, val_data

train_data, val_data = split_data(data)

# Convert data to TensorFlow tensors
train_data_tf = tf.convert_to_tensor(train_data, dtype=tf.float32)
val_data_tf = tf.convert_to_tensor(val_data, dtype=tf.float32)

# Multivariate VAE
latent_dim = 2  # Latent dimension

# Encoder
input_shape = train_data.shape[1]
inputs = Input(shape=(input_shape,))
x = Dense(64, activation='relu')(inputs)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), dtype=tf.float32)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# Decoder
latent_inputs = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(latent_inputs)
outputs = Dense(input_shape, activation='linear')(x)

decoder = Model(latent_inputs, outputs, name='decoder')

# VAE
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

# Loss
reconstruction_loss = mse(inputs, outputs)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(train_data_tf, epochs=50, batch_size=32, validation_data=(val_data_tf,))

# Generate augmented data
def generate_augmented_data(data, num_samples=100):
    latent_samples = np.random.normal(size=(num_samples, latent_dim))
    latent_samples_tf = tf.convert_to_tensor(latent_samples, dtype=tf.float32)
    generated_data = decoder.predict(latent_samples_tf)

    # Check for NaN values
    if np.any(np.isnan(generated_data)):
        # Remove NaN values
        generated_data = generated_data[~np.isnan(generated_data)]

    return generated_data

augmented_data = generate_augmented_data(train_data, num_samples=1000)



Epoch 1/50
763/763 [==============================] - 8s 8ms/step - loss: nan - val_loss: nan
Epoch 2/50
763/763 [==============================] - 8s 10ms/step - loss: nan - val_loss: nan
Epoch 3/50
763/763 [==============================] - 7s 9ms/step - loss: nan - val_loss: nan
Epoch 4/50
763/763 [==============================] - 7s 9ms/step - loss: nan - val_loss: nan
Epoch 5/50
763/763 [==============================] - 8s 10ms/step - loss: nan - val_loss: nan
Epoch 6/50
763/763 [==============================] - 6s 8ms/step - loss: nan - val_loss: nan
Epoch 7/50
763/763 [==============================] - 8s 10ms/step - loss: nan - val_loss: nan
Epoch 8/50
763/763 [==============================] - 6s 8ms/step - loss: nan - val_loss: nan
Epoch 9/50
763/763 [==============================] - 8s 10ms/step - loss: nan - val_loss: nan
Epoch 10/50
763/763 [==============================] - 6s 8ms/step - loss: nan - val_loss: nan
Epoch 11/50
763/763 [==============================] - 

In [ ]:
augmented_data = generate_augmented_data(train_data, num_samples=1000)


32/32 [==============================] - 0s 3ms/step
